#Instalar librerias
Las librerias para calculo matricial o simbolico como numpy y scipy ya estan instalada en el entorno de Colab, pero debemos instalar la libreria de robotica 

In [ ]:
!pip install roboticstoolbox-python

In [2]:
# Si se instala desde interno local usar:
!conda install -c conda-forge roboticstoolbox-python

/bin/bash: conda: command not found


Es necesario actualizar numpy a su ultima version para que sea compatible con el roboticstoolbox, tambien hay que reiniciar el ambiente.

In [3]:
!pip install numpy --upgrade

De esta forma puedes verificar si el numpy quedo actualizado, la ultimar version es la 1.21.5

In [4]:
import numpy as np
print(np.__version__)
print(np.__path__)

1.21.5
['/usr/local/lib/python3.7/dist-packages/numpy']


#Importar librerias
el roboticstoolbox internamiente importa todas las demas dependencias, se importa con el alias de "rtb" para que sea mas facil de usar, pero se puede usar cualquier otro alias.

In [5]:
#%matplotlib
#%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
import spatialmath as sm
#import numpy as np #---->importar si no se ha importado todavia
from IPython.display import HTML, Image


#Crear un robot
Existen diferentes formas de crear un robot con la libreria. Lo mas facil es usar alguno de los robots ya definidos para explorar un poco las capacidades de la libreria, tambien se puede crear un robot desde cero con los parametros DH (Denavit-Hartenberg).

##Importar un robot de la libreria
En de la carpeta *usr/local/lib/python3.7/dict-packages/robotics-toolbox-python/roboticstoolbox/models/DH/* se encuentra los robots que ya estan listos para simular a partir de una presentacion DH.
En el toolbox, no solo se usa DH para la representacion del robot, tambien existen representaciones modenas como URDF y ETS.
Para ver el listado de robots y su tipo de representacion se puede usar la funcion `models.list()`

In [ ]:
rtb.models.list()

Con esta lista, sabemos que para cargar el modelo del robot Puma 560 de fabricante Unimation basta con crear una instancia de la clase rtb.models.DH.Puma560(). mas adelante veremos que podemos crear nuevos robots como subclases de la clase DHRobot.

In [7]:
p560 = rtb.models.DH.Puma560()

Para ver la configuracion cinematica del robot, es decir la matriz DH, basta con usar la funcion `print()`. Como respuesta se optiene informacion basica del robot, la matriz DH y una tabla de posiciones como atributos del robot.

In [ ]:
print(p560)

### Actividad 1
responder:


*   ¿cuantos DOF tiene el robot?
*   ¿que tipo de articulaciones tiene?
*   ¿que significa el 0.6718 de la segunda columna primera fila?
*   ¿que significa el 90.0 de la cuarta columna primera fila?












Como mencionamos, los modelos de los robot en este toolbox son subclases se *DHRobot*. Esta clase tiene diferentes metodos y atributos que nos permiten inteactuar con el robot.

##Graficar el robot en una posicion
Para ver la confoguracion que el robot va a tomar a partir de sus posiciones **articulares** podemos usar el metodo plot() entregando como argumento de entrada una posicion articular.

Probemos graficar el robot en la posicion qz la cual es un atributo.


In [ ]:
%matplotlib notebook
print(p560.qz)  #ver los valores de la posicion articular qz
p560.plot(p560.qz, movie = "pj.gif")

###Actividad 2
graficar el robot en las posiciones qr, qs y qn

In [10]:
#su respuesta

##Simular el movimiento de robot
Para visualizar el movimiento de el robot se debe crear una trajectoria que sera un arreglo de posiciones articulares por las que pase el robot entre dos posiciones (inicial y final). Para esto usamos la Clase `tools.trajectory.jtraj` la cual tiene como atributo `q` el arreglo de posiciones articulares.


In [11]:
qt = rtb.tools.trajectory.jtraj(p560.qz, p560.qn, 50)
#qt.q #descomentar si quiere ver los valores

Ahora, con el objeto de trajectorias creado, se lo pasamos como argumento al metodo `plot()` adicionando el argumento dt que sera el paso de tiempo entre cada punto.

In [ ]:
#%matplotlib notebook
p560.plot(qt.q, dt = 0.05, movie = "2.gif")

Tambien se puede usar la funcion `qplot()` que permite ver la evolucion de las posiciones articulares en funcion del tiempo.

In [ ]:
%matplotlib inline
rtb.tools.trajectory.qplot(qt.q, block=False)

In [ ]:
p560.qrandom

###Actividad 3
Utilice la funcion qplot() con 3 nuevas trayectorias. Puede utilizar el atributo qrandom del robot para generar posiciones articulares aleatorias.

In [15]:
%matplotlib inline
#su respuesta

##Transformaciones
Internamente, los parametros de la matriz DH son usandos para calcular las matrices de transformacion homogenea (MTH). Para ver esta matriz, podemos usar el metodo `A()` entregando como argumento la articulacion hasta la cual queremos calcular la MTH y la posicion articular.

In [ ]:
MTH = p560.A(5,p560.qz) #5 es la 6ta articulacion porque se cuenta desde cero
MTH

Si quisieramos obtener la localizacion de codo del robot con respecto a la base, basta con calcular la MTH para la tercera articulacion.

In [ ]:
MTH_codo = p560.A(2,p560.qz) 
MTH_codo

###Actividad 4
Recordemos que esta matriz contiene la representacion de la localizacion del robot para la posicion articular evaluada, `qz` en el ejemplo anterior.
De acuerdo a esto, ¿Cual seria la posicion y orientacion del robot para esta MTH?

Compruebelo con la grafica del robot y repita la transformacion para la posicion articular `qs`.

In [18]:
#Su respuesta

#Cinematica Directa
El modelo cinematico directo permite calcular la localizacion del TCP en funcion de las posiciones articulares del robot. 

Con el toolbox podemos usar el metodo `fkine()` "forward kinematics" para resolcer la cinematica directa entregando como argumento las posiciones articulares.

In [ ]:
#fkine es un metodo de la clase DHRobot
pose = p560.fkine(p560.qs)
pose

El metodo  `fkine()` tambien recibe como entrada un arreglo de varias posiciones articulares y devuelve un arreglo de MTHs

In [ ]:
#fkine es un metodo de la clase DHRobot
pose_array = p560.fkine(np.array([p560.qs,p560.qz]))
pose_array


La MTH es un objeto de tipo `spatialmath.pose3d.SE3`. Esto nos permite acceder a metodos y atributos que nos facilitan el analisis de la localizacion del robot.

In [ ]:
#ver el tipo de objeto 
print(type(pose))

In [ ]:
#representar la orientacion en angulos de euler
print(pose.eul())

In [ ]:
#Sacar solo la matriz de rotacion
print(pose.R)

In [ ]:
#Sacar solo la posicion
print(pose.t)

In [ ]:
#Graficar el sistema coodenado del TCP
%matplotlib inline
print(pose.plot(color='red', label='2'))

###Actividad 5
Desarrolle una funcion que que genere una trayectoria que pase por n posiciones articulares definidas en un arreglo o lista.

In [ ]:
q_list = [p560.qn,p560.qz,p560.qn,p560.qrandom]
def TY(q):
  for i in range(len(q)-1):
    qt = rtb.tools.trajectory.jtraj(q[i], q[i+1], 10)
    if i == 0 :  
      q_traj = qt.q
    else:
      q_traj = np.concatenate((q_traj,qt.q),axis=0)
  return q_traj

gg = TY(q_list)
%matplotlib notebook
p560.plot(gg, movie = "pj.gif")

#Cinematica inversa
La cinematica inversa se puede resolver de muchas formas con el toolbox, sin embargo, algunas configuraciones de robot no tienen solucion analitica y se debe recurir a metodos numerico. 
Vamos a ver algunas forma para el robot PUMA.

El primer metodo a utilizar sera de forma iterativa o numerica, para esto, la clase DHRobot ya cuenta con funciones que "intentan" resolver la cinematica inversa pero no siempre dan un resultado valido o incluso dan un error.

Las funciones para calculo numerico del la cinematica inversa son:

*   `ikine_LM`: Por metodo Levenberg-Marquadt
*   `ikine_min` : Por metodo de minimizacion
*   `ikine_global`: Por diferentes metodos ("basinhopping", "brute" , "differential_evolution", "shgo", "dual_annealing")

Todas las funciones requieren como primer argumento un objeto de tipo SE3 que contiene la localizacion del robot (posicion + orientacion). tenemos dos formas de crear este objeto SE3.
 

1. por medio de la cinematica directa: cuando se usa la funcion fkine, la respuesta es la localizacion del robot en formato SE3

In [ ]:
q_random = p560.qrandom
T1 = p560.fkine(q_random)
print(T1)  #ver el objeto en formato de matriz
print(type(T1))  #ver el tipo de objeto
print("matriz de rotacion")
print(T1.R)
print("rotacion como angulos de euler")
print(T1.eul())
print("vector de posicion")
print(T1.t)

2. Crear el objeto a partir de una posicion y rotacion conocida

In [ ]:
from spatialmath import SE3
T2 = SE3(1, 0.2, 0.5) * SE3.RPY([0.1, 0.2, 0.3], order='xyz')  #este en una forma, usando rotaciones individiales para la rotacion.
print(T2)


Ahora, podemos usar los metodos mencionado para calcular la cinamatica inversa del robot PUMA. adicional, podemos agregar la etiqueda %%time para ver el tiempo de ejecucion


In [ ]:
%%time
print(p560.ikine_LM(T1))

En la celda anterior, podemos ver que la respuesta del metodo `ikine_LM` no solo es la solucion, sino tambien informacion sobre el proceso iterativo tal como el numero de iteraciones y el residuo. Si queremos guardar estos datos debemos asignarlos a una variable:

In [ ]:
%%time
q_1 , s, r , it, res = p560.ikine_LM(T1)
print(q_1)  #q_1 es una lista con los valores articulares

###Actividad 6
Cree un programa que evalue si la cinematica inversa fue correcta y en caso de fallas muestre porque, en caso de exito grafique el robot en esa posicion.

Pruebe su codigo con `T2`y evalue que pudo pasar.

In [ ]:
#Su respuesta

###Actividad 7
Pruebe las funciones `ikine_min` e `ikine_global ` para una nueva localizacion del robot generada aleatoriamente.

¿cual de los 3 metodos tiene mejor rendimiento?

En caso de que no llegar a una solucion, pruebe aumentar el limite de iteraciones agregando el argumento `ilimit=500`.

In [ ]:
p560.ikine_global(T=T1,method="brute")

##Resolver la cinematica inversa para una trajectoria
En la practica, es muy comun que se requiera resolver la cinematica inversa no para un solo punto, sino para varios puntos que formen una trajectora, por ejemplo una linea o un circulo. 

Para facilitar esto, los metodos anteriormente visto permiten que se entregue como argumento un arreglo de localizaciones y dando como respuesta un arreglo de posiciones articulares.

primero para generar una trajectoria en linea recta de la herramienta del robot, se puede usar la funcion ctraj, que funciona de forma similar a jtraj pero para las localizaciones.

In [84]:
T1 = p560.fkine(p560.qrandom)
T2 = p560.fkine(p560.qrandom)
t = np.arange(0, 2, 0.05)  #arreglo np de tiempos
Ts = rtb.tools.trajectory.ctraj(T1, T2, t)  #genera varias localizaciones intermedias entre T1 y T2, una para cada tiempo

CPU times: user 571 ms, sys: 624 ms, total: 1.2 s
Wall time: 956 ms


In [86]:
%%time
Qs = p560.ikine_LM(Ts) #Calcula la IK para cada localizacion, esto puede tardar un tiempo

CPU times: user 545 ms, sys: 641 ms, total: 1.19 s
Wall time: 1.02 s


Tenga en cuenta que la cinematica inversa para toda la trayectoria puede fallar porque se generaron entre puntos aleatorios.

##Cinematica inversa de forma analitica
para el robot PUMA ya se cuenta con una solicion analitica, se puede aplicar de la misma forma que los metodo anteriores pero usando `ikine_a`.





In [87]:
%%time
Qsa = p560.ikine_a(Ts)
#Q1 = p560.ikine_a(T1)

CPU times: user 41.1 ms, sys: 0 ns, total: 41.1 ms
Wall time: 49.9 ms


Recordemos que no siempre se cuenta con la solucion analitica, pero en el ejemplo anterior es evidente que el rendimiento computacional es mucho mejor que con metodos numericos.

##Actividad 9
Cree una funcion que a partir de un punto dentro del volumen de trabajo del robot, cree una trajectoria cuadrada en un plano horizontal dado el largo del lado del cuadrado. Puede ser sobre los ejes *x* y *y*.
Despues resuelva la cinematica inversa para los puntos de esta trayectoria.